# RFM Segmentation
Synthetic dataset about custumer segmentation with 500k rows.

### Variables

customer_id = The unique customer ID

qtt_order = The order of customers

total_spent = The total spent of the customers

last_order = Last order time of the customers

In [15]:
import pandas as pd
import datetime as dt

pd.set_option("display.width", 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [30]:
df= pd.read_csv("/kaggle/input/customer-segmentation-rfm/customer_segmentation_500k.csv")
df.head()

,customer_id,qtt_order,total_spent,last_order
0,0,272,90067.35,2022-10-23 09:00:00
1,1,825,194707.21,2022-09-28 09:00:00
2,2,132,224159.97,2022-06-07 19:00:00
3,3,62,287130.72,2022-01-25 09:00:00
4,4,4,71469.24,2022-07-01 18:00:00


### Data Preperation for RFM Analysis

In [32]:
def check_df(dataframe):
    print("############ Head ##############")
    print(dataframe.head())
    print("############ Tail ##############")
    print(dataframe.tail())
    print("############ Shape ##############")
    print(dataframe.shape)
    print("############ N/A ##############")
    print(dataframe.isnull().sum())
    print("############ Types ##############")
    print(dataframe.dtypes)
    print("############ Quantile ##############")
    print(dataframe.describe().T)

check_df(df)

############ Head ##############
   customer_id  qtt_order  total_spent           last_order
0            0        272     90067.35  2022-10-23 09:00:00
1            1        825    194707.21  2022-09-28 09:00:00
2            2        132    224159.97  2022-06-07 19:00:00
3            3         62    287130.72  2022-01-25 09:00:00
4            4          4     71469.24  2022-07-01 18:00:00
############ Tail ##############
        customer_id  qtt_order  total_spent           last_order
499995       499995        482    253253.34  2022-09-25 22:00:00
499996       499996         44    122348.74  2022-07-09 01:00:00
499997       499997         83     46623.55  2022-07-15 16:00:00
499998       499998         49    140711.18  2022-10-01 08:00:00
499999       499999         20    158831.36  2022-05-16 01:00:00
############ Shape ##############
(500000, 4)
############ N/A ##############
customer_id    0
qtt_order      0
total_spent    0
last_order     0
dtype: int64
############ Types ######

In [33]:
df["last_order"]= pd.to_datetime(df["last_order"])
df.dtypes
df["customer_id"] = df["customer_id"].astype(str)
df["last_order"].max()
today = dt.datetime(2023, 1, 24)# analysis day

### RFM Analysis

In [34]:
#creating recency, frequency, monetary
rfm = df.groupby("customer_id").agg({"last_order": lambda x: (today - x.max()).days,
                                    "qtt_order": lambda x : x.nunique(),
                                    "total_spent": lambda x: x.sum()})
rfm.columns = ['recency', 'frequency', 'monetary']
rfm.head()

,recency,frequency,monetary
customer_id,,,
0,92,1,90067.35
1,117,1,194707.21
10,246,1,178569.43
100,309,1,26260.98
1000,250,1,67285.83


In [38]:
#creating recency score, frequency score, monetary score and RFM score
rfm["recency_score"]= pd.qcut(rfm["recency"], 5, labels = [5,4,3,2,1])
rfm["frequency_score"]= pd.qcut(rfm["frequency"].rank(method = "first"), 5, labels = [1,2,3,4,5])
rfm["monetary_score"]= pd.qcut(rfm["monetary"], 5, labels = [1,2,3,4,5])
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
customer_id,,,,,,,
0,92,1,90067.35,5,1,2,51
1,117,1,194707.21,5,1,4,51
10,246,1,178569.43,3,1,3,31
100,309,1,26260.98,2,1,1,21
1000,250,1,67285.83,3,1,2,31


### Creating & Analysing RFM Segments

In [39]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [40]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
customer_id,,,,,,,,
0,92,1,90067.35,5,1,2,51,new_customers
1,117,1,194707.21,5,1,4,51,new_customers
10,246,1,178569.43,3,1,3,31,about_to_sleep
100,309,1,26260.98,2,1,1,21,hibernating
1000,250,1,67285.83,3,1,2,31,about_to_sleep


In [12]:
rfm.index = rfm.index.astype(int)


## Function of RFM Analysis

In [41]:
def rfm_analysis(dataframe, csv = False):
    dataframe["last_order"] = pd.to_datetime(dataframe["last_order"])
    dataframe.dtypes
    dataframe["customer_id"] = dataframe["customer_id"].astype(str)
    dataframe["last_order"].max()
    today = dataframe.datetime(2023, 1, 24)

    rfm = dataframe.groupby("customer_id").agg({"last_order": lambda x: (today - x.max()).days,
                                         "qtt_order": lambda x: x.nunique(),
                                         "total_spent": lambda x: x.sum()})
    rfm.columns = ['recency', 'frequency', 'monetary']
    rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                        rfm['frequency_score'].astype(str))
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }
    rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
    rfm = rfm[["recency", "frequency", "monetary", "segment"]]
    rfm.index = rfm.index.astype(int)
    if csv:
        rfm.to_csv("rfm.csv")

    return rfm